In [ ]:
#!pip install matplotlib

In [ ]:
#!pip install scikit-learn

In [ ]:
# import global modules
import os
import re
import sys
import time
import json
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
from yaml import safe_load
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_utils = pth_project / 'utils'
pth_queries = pth_project / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, pth_project.as_posix())
d_config = safe_load(pth_creds.open())

# import local modules
from utils.gcp import connect_bq_services, connect_pandas_bq_services
from utils.extract import extract_bq_data
from utils.modeling import process_features, extract_stats

In [ ]:
bq_client = connect_bq_services(d_config['gcp-project-name'])

In [ ]:
%load_ext autoreload
%autoreload 2

#### Extract data

In [ ]:
# extract trainning data
sql = f"""
  select *
    from `divg-team-v03-pr-de558a.nba_product_reco_model.nba_training_dataset_v9` #Jan to Oct
"""
df_train = extract_bq_data(bq_client, sql)
df_train.shape

In [ ]:
# extract validation data
sql = f"""
  select *
    from `divg-team-v03-pr-de558a.nba_product_reco_model.nba_val_dataset_v9` #Nov 
"""
df_validation = extract_bq_data(bq_client, sql)
df_validation.shape

In [ ]:
# extract test data
sql = f"""
  select *
    from `divg-team-v03-pr-de558a.nba_product_reco_model.nba_test_dataset_v9` #Dec 
"""
df_test = extract_bq_data(bq_client, sql)
df_test.shape

#### Process data

In [ ]:
#!pip install category-encoders
#import category_encoders as ce

In [ ]:
# create target output
# d_target_mapping = {
#     scenario: i
#     for i, scenario in enumerate(df_train['model_scenario'].unique())
# }

d_target_mapping = {
 'sing_acquisition': 0,
 'shs_acquisition': 1,
 'tos_acquisition': 2,
 'wifi_acquisition': 3,
 'ttv_acquisition': 4,
 'sws_acquisition': 5,
 'hsic_acquisition': 6,
 'lwc_acquisition': 7,
 'hpro_acquisition': 8,
 'whsia_acquisition': 9
}

d_target_mapping

In [ ]:
# load features metadata
d_features_metadata = safe_load((pth_utils / 'parameters' / 'acquisition_features_v5.yaml').open())

In [ ]:
# process training data
df_train_processed = process_features(df_train, d_features_metadata, 'model_scenario', d_target_mapping)
df_validation_processed = process_features(df_validation, d_features_metadata, 'model_scenario', d_target_mapping)
df_test_processed = process_features(df_test, d_features_metadata, 'model_scenario', d_target_mapping)

In [ ]:
print(df_train_processed.shape)
print(df_validation_processed.shape)
print(df_test_processed.shape)

In [ ]:
# l_cat_features = [
#     'acct_cr_risk_txt',
#     'acct_ebill_ind',
#     'cust_cr_val_txt',
#     'cust_pref_lang_txt',
#     'cust_prov_state_cd'
# ]

In [ ]:
# df_train_all_num = df_train[df_train.select_dtypes(exclude=['object']).columns]
# df_validation_all_num = df_validation[df_train_all_num.columns]

In [ ]:
# for col in l_cat_features:
#     df_train_all_num[col] = df_train[col]
#     df_validation_all_num[col] = df_validation[col]

# for col in l_cat_features:
#     df_train_processed[col] = df_train[col]
#     df_validation_processed[col] = df_validation[col]

In [ ]:
# df_train_all_num['fsa'] = df_train_processed['fsa']
# df_validation_all_num['fsa'] = df_validation_processed['fsa']

In [ ]:
# df_train_all_num = df_train_all_num.fillna(0)
# df_validation_all_num = df_validation_all_num.fillna(0)

#### Split data

In [ ]:
# Separate the features and target variable
ban_train = df_train[['ban', 'lpds_id']]
X_train = df_train_processed.drop(columns='target')
y_train = df_train_processed['target']

ban_val = df_validation[['ban', 'lpds_id']]
X_val = df_validation_processed.drop(columns='target')
y_val = df_validation_processed['target']

ban_test = df_test[['ban', 'lpds_id']]
X_test = df_test_processed.drop(columns='target')
y_test = df_test_processed['target']

In [ ]:
# # Separate the features and target variable
# X = df_train_processed.drop(columns='target')
# #X = df_train_all_num.drop(columns=['cust_src_id', 'ban', 'ban_src_id', 'lpds_id', 'label', 'ban_1'])
# y = df_train_processed['target']

# X_val = df_validation_processed.drop(columns='target')
# #X_val = df_validation_all_num.drop(columns=['cust_src_id', 'ban', 'ban_src_id', 'lpds_id', 'label', 'ban_1'])
# y_val = df_validation_processed['target']

In [ ]:
#X = X.fillna(0)
#X_val = X_val.fillna(0)

In [ ]:
# # train test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# use target encoding to encode two categorical features
#enc = ce.TargetEncoder(cols=['fsa'])
#enc = ce.TargetEncoder(cols=l_cat_features + ['fsa'])

In [ ]:
# transform the datasets
# X_train = enc.fit_transform(X_train, y_train)
# X_test = enc.transform(X_test)
# X_val = enc.transform(X_val)

#### Sampling

In [ ]:
#!pip install imblearn

In [ ]:
#from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
# Apply random oversampling to balance the dataset
#sampling = RandomOverSampler(random_state=42, sampling_strategy=sampling_strategy)
# sampling = SMOTE(random_state=42)

# X_train_resampled, y_train_resampled = sampling.fit_resample(X_train.astype('float'), y_train)

In [ ]:
#y_train_resampled.value_counts()

#### XGBoost

In [ ]:
#!pip install xgboost
import xgboost as xgb

##### Tunning

In [ ]:
#!pip install optuna
#import optuna

In [ ]:
# # Define the objective function for Optuna
# def objective(trial):
#     # Define the hyperparameters to tune
#     params = {
#         'objective': 'multi:softmax',
#         'eval_metric': 'mlogloss',
#         'num_class': 9,
#         'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
#         'max_depth': trial.suggest_int('max_depth', 3, 10),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#     }
    
#     # Train the XGBoost model with the current set of hyperparameters
#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dval = xgb.DMatrix(X_val, label=y_val)
#     model = xgb.train(params, dtrain, num_boost_round=100)

#     # Make predictions on the validation set
#     y_pred = model.predict(dval)

#     # Calculate the accuracy score
#     accuracy = accuracy_score(y_val, y_pred)

#     return 1 - accuracy

In [ ]:
# Define the Optuna study
#study = optuna.create_study(direction='minimize')

In [ ]:
# Start the optimization process
#study.optimize(objective, n_trials=100)

In [ ]:
# Get the best hyperparameters and the best score
# best_params = study.best_params
# best_score = study.best_value

# print("Best Hyperparameters:", best_params)
# print("Best Score:", best_score)

In [ ]:
# best_params.update({
#     'objective': 'multi:softmax',
#     'eval_metric': 'mlogloss',
#     'num_class': 9
# })

In [ ]:
#best_params

In [ ]:
# xgb_model = xgb.XGBClassifier(
#     best_params
# )

In [ ]:
# Train the best model with the best hyperparameters
#best_model = xgb.train(best_params, xgb.DMatrix(X_train, label=y_train), num_boost_round=100)

In [ ]:
# # hyperparamètres
# params = {
#     "colsample_bytree": np.arange(0.5, 1.0, 0.1),
#     "gamma": uniform(0, 0.5),
#     "learning_rate": [0.01, 0.1, 0.2, 0.3], # default 0.1 
#     "max_depth": randint(3, 7), # default 3
#     "n_estimators": randint(100, 200), # default 100
#     "subsample": np.arange(0.5, 1.0, 0.1),
#     "min_child_weight" : range(1,6,2),
#     "objective": 'multi:softproba'
# }

In [ ]:
# search = RandomizedSearchCV(
#     xgb_model, 
#     param_distributions=params, 
#     random_state=42, 
#     n_iter=20, 
#     cv=5, 
#     verbose=3, 
#     n_jobs=1,
#     scoring='accuracy', 
#     return_train_score=True
# )

In [ ]:
#search_result = search.fit(X_train, y_train)

In [ ]:
# print("Best parameters:", search_result.best_params_)
# print("Best score: ", search.best_score_)

# xgb_best_model = search_result.best_estimator_

##### Training

In [ ]:
xgb_model = xgb.XGBClassifier()

In [ ]:
xgb_model = xgb.XGBClassifier(
    learning_rate=0.1,
    n_estimators=250,
    max_depth=8,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softproba',
    num_class=3, 
    eval_metric='mlogloss', 
    nthread=4,
    scale_pos_weight=1,
    seed=27,
    verbose=1
)

In [ ]:
# #xgb_model.fit(X_train_resampled, y_train_resampled)
# xgb_model.fit(X_train, y_train)

In [ ]:
# n= 3
# probabilities =  xgb_model.predict_proba(X_val)
# results_ranked = np.argsort(-probabilities, axis=1)
# display(extract_stats(n, results_ranked, y_val, d_target_mapping))

In [ ]:
# n= 3
# probabilities =  xgb_model.predict_proba(X_test)
# results_ranked = np.argsort(-probabilities, axis=1)
# display(extract_stats(n, results_ranked, y_test, d_target_mapping))

In [ ]:
# y_pred = xgb_model.predict_proba(X_test, ntree_limit=xgb_model.best_iteration)

# y_pred_0 = y_pred[:, 0]
# y_pred_1 = y_pred[:, 1]
# y_pred_2 = y_pred[:, 2]
# y_pred_3 = y_pred[:, 3]
# y_pred_4 = y_pred[:, 4]
# y_pred_5 = y_pred[:, 5]
# y_pred_6 = y_pred[:, 6]
# y_pred_7 = y_pred[:, 7]
# y_pred_8 = y_pred[:, 8]
# y_pred_9 = y_pred[:, 9]

# # #predictions on X_test
# # y_pred = xgb_model.predict_proba(X_test, ntree_limit=xgb_model.best_iteration)[:, 1]

# df_ban_test = ban_test
# df_test_exp = df_ban_test.join(X_test) 
# df_test_exp['y_test'] = y_test
# df_test_exp['y_pred_proba_0'] = y_pred_0
# df_test_exp['y_pred_proba_1'] = y_pred_1
# df_test_exp['y_pred_proba_2'] = y_pred_2
# df_test_exp['y_pred_proba_3'] = y_pred_3
# df_test_exp['y_pred_proba_4'] = y_pred_4
# df_test_exp['y_pred_proba_5'] = y_pred_5
# df_test_exp['y_pred_proba_6'] = y_pred_6
# df_test_exp['y_pred_proba_7'] = y_pred_7
# df_test_exp['y_pred_proba_8'] = y_pred_8
# df_test_exp['y_pred_proba_9'] = y_pred_9
# # df_test_exp['y_pred_proba_10'] = y_pred_10
# # df_test_exp['y_pred_proba_11'] = y_pred_11
# # df_test_exp['y_pred_proba_12'] = y_pred_12

# import datetime

# # Get the current timestamp
# current_timestamp = datetime.datetime.now()

# df_test_exp.to_csv(f"gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_test_exp_{current_timestamp}.csv")

In [ ]:
# # Retrieve feature importance scores
# importance_scores = xgb_model.feature_importances_
# feature_names = X_train.columns

# # Sort feature importance scores and feature names in descending order
# sorted_indices = importance_scores.argsort()
# sorted_scores = importance_scores[sorted_indices][-30:]
# sorted_names = feature_names[sorted_indices][-30:]

# # Plot feature importance
# plt.figure(figsize=(8, 8))
# plt.barh(range(len(sorted_scores)), sorted_scores)
# plt.yticks(range(len(sorted_scores)), sorted_names)
# plt.xlabel('Importance Score')
# plt.ylabel('Features')
# plt.title('Feature Importance')
# plt.show()

In [ ]:
list_learning_rate = [0.05, 0.1, 0.15]
list_n_estimators = [200, 300]
list_max_depth = [10, 20] 
list_objective = ['multi:softproba']
list_params = [] 
list_results = []
i = 0

for learning_rate in list_learning_rate: 
    for n_estimators in list_n_estimators: 
        for max_depth in list_max_depth: 
            
            i += 1 

            xgb_model = xgb.XGBClassifier(
                learning_rate=learning_rate,
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_child_weight=1,
                gamma=0,
                subsample=0.8,
                colsample_bytree=0.8,
                objective='multi:softproba',
                num_class=3, 
                eval_metric='mlogloss', 
                nthread=4,
                scale_pos_weight=1,
                seed=27,
                verbose=1
            )

            #xgb_model.fit(X_train_resampled, y_train_resampled)
            xgb_model.fit(X_train, y_train)

            n= 3
            probabilities =  xgb_model.predict_proba(X_val)
            results_ranked = np.argsort(-probabilities, axis=1)
            
            df_stat = extract_stats(n, results_ranked, y_val, d_target_mapping)
            list_results.append(df_stat)
            
            params = {'index': i,
                      'learning_rate': learning_rate, 
                      'n_estimators': n_estimators, 
                      'max_depth': max_depth}
            
            list_params.append(params)

In [ ]:
for idx, result in enumerate(list_results): 
    print(f'params: {list_params[idx]}')
    print(result)
    print(' ')
          
    lr = list_params[idx]['learning_rate']
    ne = list_params[idx]['n_estimators']
    md = list_params[idx]['max_depth']
          
    result.to_csv(f"gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_stats_{lr}_{ne}_{md}")


In [ ]:
# Define the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [50, 70, 100, 200, 250],
    'max_depth': [5, 7, 10], 
    'objective': 'multi:softproba'
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Evaluate the model on the test set
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)


#### CatBoost

In [ ]:
#!pip install catboost

In [ ]:
#from catboost import CatBoostClassifier

In [ ]:
#cat_model = CatBoostClassifier()

In [ ]:
#cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=10) 

#### LightGBM

In [ ]:
#!pip install lightgbm

In [ ]:
#from lightgbm import LGBMClassifier

In [ ]:
#lgbm = LGBMClassifier(learning_rate=0.1, n_estimators=100, num_leaves=31)

In [ ]:
#lgbm.fit(X_train_resampled, y_train_resampled)

In [ ]:
#lgbm.fit(X_train, y_train)

#### Results

In [ ]:
# d_models_mapping = {
#     'XGBClassifier': xgb_model,
#     #'LGBMClassifier': lgbm,
#     'CatBoost': cat_model,
#     # 'OneVsRestClassifier': xgb_best_model,
#     # 'OneVsOneClassifier': xgb_best_model,
#     #'RandomForest': rf_model,
#     #'SVM': svm,
#     #'Logistic Regression': lr,
#     # 'XGBClassifier': xgb_best_model,
#     # 'DeepLearning': xgb_best_model,
# }

In [ ]:
n= 3
probabilities =  xgb_model.predict_proba(X_val)
results_ranked = np.argsort(-probabilities, axis=1)
display(extract_stats(n, results_ranked, y_val, d_target_mapping))

In [ ]:
print(len(probabilities))
print(len(y_val))

In [ ]:
# y_pred = xgb_model.predict_proba(X_val)

y_pred_0 = probabilities[:, 0]
y_pred_1 = probabilities[:, 1]
y_pred_2 = probabilities[:, 2]
y_pred_3 = probabilities[:, 3]
y_pred_4 = probabilities[:, 4]
y_pred_5 = probabilities[:, 5]
y_pred_6 = probabilities[:, 6]
y_pred_7 = probabilities[:, 7]
y_pred_8 = probabilities[:, 8]
y_pred_9 = probabilities[:, 9]

df_val_exp = X_val
df_val_exp['y_test'] = y_val
df_val_exp['y_pred_proba_0'] = y_pred_0
df_val_exp['y_pred_proba_1'] = y_pred_1
df_val_exp['y_pred_proba_2'] = y_pred_2
df_val_exp['y_pred_proba_3'] = y_pred_3
df_val_exp['y_pred_proba_4'] = y_pred_4
df_val_exp['y_pred_proba_5'] = y_pred_5
df_val_exp['y_pred_proba_6'] = y_pred_6
df_val_exp['y_pred_proba_7'] = y_pred_7
df_val_exp['y_pred_proba_8'] = y_pred_8
df_val_exp['y_pred_proba_9'] = y_pred_9

df_val_exp.to_csv("gs://divg-groovyhoon-pr-d2eab4-default/downloads/df_val_exp.csv")

#### Features Importance

In [ ]:
# Retrieve feature importance scores
importance_scores = xgb_model.feature_importances_
feature_names = X_train.columns

# Sort feature importance scores and feature names in descending order
sorted_indices = importance_scores.argsort()
sorted_scores = importance_scores[sorted_indices][-30:]
sorted_names = feature_names[sorted_indices][-30:]

# Plot feature importance
plt.figure(figsize=(8, 8))
plt.barh(range(len(sorted_scores)), sorted_scores)
plt.yticks(range(len(sorted_scores)), sorted_names)
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.title('Feature Importance')
plt.show()

#### Correlation

In [ ]:
!pip install seaborn
import seaborn as sns

In [ ]:
# seaborn heatmap around the correlation matrix.
plt.figure(figsize=[40,40])
sns.heatmap(
    X.corr(numeric_only=True), 
    annot=True, 
    fmt='.2f',
    square=True, 
    cmap='vlag', 
    center=0, 
)